In [1]:
import random
import tensorflow as tf
import numpy as np
import pandas as pd

import nltk
from gensim.models import Word2Vec
from gensim.utils import tokenize

from collections import defaultdict

C:\dev\Anaconda3\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# Hyper Parameter

EPOCHS = 50
NUM_WORDS = 30000

In [3]:
origin_path = 'c:/dev/dataset/wikihowAll_pre.csv'

origin_data = pd.read_csv(origin_path)
origin_data = origin_data.drop(origin_data.columns[0], axis=1)
origin_data = origin_data.dropna()
origin_data

,headline,title,text
0,Keep related supplies in the same area.Make an...,How to Be an Organized Artist1,If you're a photographer keep all the necessa...
1,Create a sketch in the NeoPopRealist manner of...,How to Create a Neopoprealist Art Work,See the image for how this drawing develops s...
2,Get a bachelor’s degree.Enroll in a studio-bas...,How to Be a Visual Effects Artist1,It is possible to become a VFX artist without...
3,Start with some experience or interest in art....,How to Become an Art Investor,The best art investors do their research on t...
4,Keep your reference materials sketches article...,How to Be an Organized Artist2,As you start planning for a project or work y...
...,...,...,...
214289,Consider changing the spelling of your name.Av...,How to Pick a Stage Name3,If you have a name that you like you might fi...
214290,Try out your name.Don’t legally change your na...,How to Pick a Stage Name4,Your name might sound great to you when you s...
214291,Understand the process of relief printing.Exam...,How to Identify Prints1,Relief printing is the oldest and most tradit...
214292,Understand the process of intaglio printing.Lo...,How to Identify Prints2,"Intaglio is Italian for ""incis­ing"" and corre..."


In [4]:
x, y = origin_data['text'], origin_data.headline

In [22]:
# tokenized dictionary
def tokenized_to_dic(df) :
    x_token = defaultdict(int)
    y_token = defaultdict(int)

    for idx in range(len(df))



([], [])

In [5]:
# encoder & decoder
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder, self).__init__()
        # 입력이 one-hot-encdoing 형식으로 들어오면 embedding을 수행을 먼저
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        # 그리고 LSTM에서 hidden state을 출력으로 던져주어야 이것을 활용해서 return을 활용할 수 있다...
        # return_sequence=True : 디코더에서 나오는 것 하나하나 알아야 하기에...
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)
        # return_sequences = True -> attention 하기 위해서

    # 인코더의 경우에는 입력이 들어오면 이것을 바탕으로 히든, 셀 스테이트를 하도록 ...
    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        # h,c는 test를 할 경우에는 다음으로 넘겨 주어야 forward  로 계산을 하면서 나갈 수 있으니..
        H, h, c = self.lstm(x)
        return H, h, c
    
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)
        
        #****Attention을 추가함!!!! 
        self.att = tf.keras.layers.Attention()
        
        # 최종 단어 단에서 어떤 것을 할 것인지..선택..
        self.dense = tf.keras.layers.Dense(NUM_WORDS, activation='softmax')

    def call(self, inputs, training=False, mask=None):
        # ***처음 들어오는 인코더 단에서 넘어오는 것들을 받고,,
        x, s0, c0, H = inputs
        x = self.emb(x)
        # ****S는 hidden state를 모두 모아둔 부분..--> 쿼리로 사용을 할 것임....--->하나 앞선 시간을 사용을 해서..
        S, h, c = self.lstm(x, initial_state=[s0, c0])
        # ***쿼리로 사용을 할 것임....--->하나 앞선 시간을 사용을 해서..(1차원을 3차원으로 확장) & 마지막 히든은 제외...해서 :-1까지..
        S_ = tf.concat([s0[:, tf.newaxis, :], S[:, :-1, :]], axis=1)
        # ****키와 value를 계산하고,,,
        A = self.att([S_, H])
        y = tf.concat([S, A], axis=-1)
        
        return self.dense(y), h, c

In [6]:
# seq2seq
class Seq2seq(tf.keras.Model):
    def __init__(self, sos, eos):
        super(Seq2seq, self).__init__()
        # 1) 가장 기본적으로 필요한 Encoder, Deoder, sos, eos에 관련된 부분을 지정을 함!!!!
        self.enc = Encoder()
        self.dec = Decoder()
        self.sos = sos
        self.eos = eos
    
    # 연결에서 구성 --> 학습을 위해서는 입력/출력 모두 알아야 하기에 x,y = input으로 받음
    # 그리고 디코더에 입력을 넣어주어야 하기에..
    def call(self, inputs, training=False, mask=None):
       # (학습 과정)
        if training is True:
            # 학습을 위해서는 입력/출력 모두 알아야 하기에 x,y = input으로 받음
            # 그리고 디코더에 입력을 넣어주어야 하기에..
            x, y = inputs
            # encoder에 입력  x를 넣어서 나온 hidden state, cell  (LSTM으로 구현이 되어서..)
            H, h, c = self.enc(x)
            # 최종 출력 y를 하도록...
            y, _, _ = self.dec((y, h, c, H))
            return y
        # (Test 과정) --> 그러니 정답  y는 없음..
        else:
            x = inputs
            H, h, c = self.enc(x)
            # 뒤에 디커더 부분에 입력을 넣어 주는 부분이 달라지게 된다!!!! --> sos  를 넣어준다.
            y = tf.convert_to_tensor(self.sos)
            y = tf.reshape(y, (1, 1))

            # 최대 seq는 64길이까지만..
            seq = tf.TensorArray(tf.int32, 64)
            
            # tf의  for loop으로 사용을 해서 최대 64까지 
            for idx in tf.range(64):
                # for 에서 처음에는 처음으로 만들어준 sos를 decoder에 입력으로 넣어주고 --> self.dec(y,h,c,H)을 해서
                # 출력인 y와 hidden state, cell state를 출력으로 준다.
                y, h, c = self.dec([y, h, c, H])
                # 가장 큰 y값에 대한 index를 얻어오게 되는 과정
                y = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32)
                # reshape를 하면서 batch를 사용을 하기 위해서...
                y = tf.reshape(y, (1, 1))
                # seq를 하나씩 받으면서 처리하기 위해서...
                seq = seq.write(idx, y)
                # eos일 때 까지 for lopp순환
                if y == self.eos:
                    break
            # 
            return tf.reshape(seq.stack(), (1, 64))

In [7]:
# implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    # 마지맞 eos는 있고, 처음 sos 는 없는 것...
    output_labels = labels[:, 1:]
    # 처음 sos는 포함이 되고, 마지막 eos는 빼고...
    shifted_labels = labels[:, :-1]
    # 그래서 위의 시프트 된 것들이 아래의 model의 학습 과정에 들어가게 된다!!!!!
    with tf.GradientTape() as tape:
        # 위의 output_labels, shifted_labels 이 들어가게 된다...
        predictions = model([inputs, shifted_labels], training=True)
        loss = loss_object(output_labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(output_labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, inputs):
    return model(inputs, training=False)


In [8]:
x_sub = list(x[:10000].apply(lambda x : x+'\n'))
y_sub = list(y[:10000].apply(lambda x : '\t'+x+'\n'))

In [9]:
num_sample = len(x_sub)

# 질문을 보고 한 번 데이터 셋을 섞어준다..
perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)
# 섞으면서 train/ test로 구분을 하기 위한 것..
train_q = list()
train_a = list()
test_q = list()
test_a = list()

# 질문과 답변에 대한 것을 돌면서...
for idx, qna in enumerate(zip(x_sub, y_sub)):
    q, a = qna
    # 5/1은 test, 5/4는 train
    if perm[idx] > num_sample//5:
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)

# 문장에서 짤라주는 것에 대한 세팅...
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')

tokenizer.fit_on_texts(train_q + train_a)

train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

# 입력은 뒤로...패딩..
x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=256)
# 출력은 앞에로 패딩..
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=64)


x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=256)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=64)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1).prefetch(1024)

In [10]:
# Create model
model = Seq2seq(sos=tokenizer.word_index['\t'],
                eos=tokenizer.word_index['\n'])

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

In [11]:
# train loop
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100))

    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch 1, Loss: 4.843564033508301, Accuracy: 37.61859130859375
Epoch 2, Loss: 4.2939252853393555, Accuracy: 39.797828674316406
Epoch 3, Loss: 4.021904945373535, Accuracy: 41.34207534790039
Epoch 4, Loss: 3.778144121170044, Accuracy: 42.70573425292969


KeyboardInterrupt: 

<PrefetchDataset shapes: ((None, 64), (None, 65)), types: (tf.int32, tf.int32)>

In [20]:
# test
for test_seq, test_labels in test_ds:
    prediction = test_step(model, test_seq)
    test_text = tokenizer.sequences_to_texts(test_seq.numpy())
    gt_text = tokenizer.sequences_to_texts(test_labels.numpy())
    texts = tokenizer.sequences_to_texts(prediction.numpy())
    print('_')
    print('Question : ', test_text)
    print('Prediction Answer: ', texts)
    print('Real Answer : ', gt_text)

_
Question :  ["its value it should be kept at low humidity and avoid being you may choose to hang it in your home but you may want to get an art collector's advice about where to hang it and how to care for it share this information with your children if it is intended as an inheritance investment they need to be well aware of how to take care of art or they may lose money or ruin the painting entirely as well as keeping tabs on the art world to understand the rise and falls of certain schools of art an can tell you how your investment is maturing they may clue you in on when you have reached your desired profit if the art you buy does not fit in your home and you will only be storing it research banks hotels and other institutions that rent fine art on a rotating basis you may be able to demand thousands of dollars per year for your art to hang in another building keep in mind you will need to make sure your art is covered by an insurance policy for loss or damage make sure the provi

_
Question :  ["about 1 8 of the tip bent over the edge of the hole formed by the nail pointing towards the toe this ensures that the nail will hold the shoe in place without the sharp tip hurting you or the horse set a block under the nail on the outer hoof wall then set the nail in the hoof or it by hitting the nail head once again with the driving hammer this gives the nail a firm grip in the horse's hoof ensuring the shoe stays in place you may use a sturdy metal tool like your in place of the block in a pinch another option is to use a special tool called a clincher with this tool all you need to do is align the jaws of the clincher over the nail tip and squeeze the handles finally give the outside of the hoof a once over with a rasp smoothing out any uneven spots and giving the hoof a nice clean finish pay special attention to the nail tips which can also be filed smooth when finished you should be able to draw a cloth over the hoof wall without it catching you may also notice th

_
Question :  ["kitchen this might be a better option than buying commercial saddle soap as the products used can be harmful to leather use protective gloves throughout the cooking process you'll be handling which is a pretty heavy duty material you want to make sure you avoid burning your skin use an apron to cook as should not come into contact with clothes you wear regularly get your ingredients you'll need two pots made of stainless steel or heat resistant glass a stainless steel spoon and some kind of jars or boxes to store the concoction you will also need beeswax pure flake soap shavings water and pure gum these products can be purchased at grocery stores hardware stores and health food stores in one pot combine 6 25 parts beeswax 10 parts pure flake and 10 parts water boil the mixture for 5 minutes stirring constantly get the second pot melt together 2 parts shavings and 10 parts water once the soap add it to the first mixture move the pot away from heat and add 15 parts pour t

_
Question :  ["this includes apples oranges strawberries and so forth if you want get some lettuce as well try not to mash the pieces together or it won't taste as good for the guinea pig you may have already put some apples and oranges in your salad but this will help bring out the flavor this step is optional note by natural this mean squeezing the juice straight out of the orange or apple do not use the orange juice or apple juice you would usually drink \n"]
Prediction Answer:  ['a cage of the cage of the cage place the cage of the cage place the cage of the cage \n']
Real Answer :  ['\tgather all of the fruit you can cut the fruit into small pieces mix the fruit pieces together add some freshly squeezed orange juice and or apple juice as dressing serve the fruit salad to your guinea pig and watch your guinea pig enjoy \n']
_
Question :  ["place it in front of the withers and slide it a little bit back until it's two or three inches in front of where you'll place the saddle rememb

_
Question :  ["her to your room in your bed take her to a quiet romantic looking place and she'll most probably get the message that you want a kiss but if she doesn't want to come to a private place with you that either means that she's not ready to kiss you it's her first time she feels uncomfortable with you or she's just nervous or shy don't force it when she's ready you will know don't hurt her feelings though tease her in a joking and funny manner and make sure that she knows that you're just joking never tease her by making fun of her if she tries to come closer to you it can signal that she's into you if she just smiles and or blushes it may mean that she likes you but she's shy nothing is worse than telling a girl she's beautiful when she knows it's not true try complimenting her on specific things like how pretty her hair is or how soft her hand feels in yours if she tries to come closer to you holds your hand and squeezes it gently smiles a lot looks at you most of the time

...

_
Question :  ['and contain moderate fat and complex carbohydrates are best to promote your general health keep your daily calorie intake around 1800 depending on how active you are you should get your calories from nutrient dense whole foods such as whole grains fruits and vegetables dairy and lean proteins exercise is important to maintaining your weight and promoting your overall health participate in low impact moderate intensity cardiovascular activities every day aim to exercise for least 30 minutes of exercise most days of the week keeping your cholesterol in check can not only help prevent jaundice but also help maintain your overall health you can manage your cholesterol through healthy diet and exercise or in other cases with prescription medication eating more soluble fiber healthy fats and foods high in omega 3 fatty acids can help manage your cholesterol foods such as lean cuts of meat low fat dairy olive oil salmon almonds oats lentils and vegetables contains these three 

_
Question :  ['because the nerve tissue beyond the damage point dies and the nerve must regenerate between healthy nerve endings a pinched nerve can occur from a number of reasons including bad posture injury arthritis spinal and or obesity nsaids treats swelling and inflamed nerves acetaminophen only treats pain be sure that these medications will not interact with any other medication for example avoid taking aspirin when on blood thinners long term nsaid use can lead to and gastric ulcers be aware of your use of these medications if a nerve was pinched rather than severed physical therapy pt is often used to repair the damage and increase strength and mobility check if your doctor will prescribe physical therapy some insurance companies will not cover pt always check with your insurance company if you have a question about you may need to wait several weeks or months after acute injury to start this phase of repair the nerve may need time to heal and regrow try de weighted training

_
Question :  ["sun is the worst for your skin try to avoid going out during these times if you do cover your hair and head with a hat or seek shade beneath a stand of trees to maximize hair and scalp protection water snow and sand all reflect uv light back at you if you visit a beach or during the late morning or afternoon wearing a hat and high spf sunscreen is especially important if you have long hair but no hat consider wearing a ponytail bun or updo unlike hairstyles with a part these will cover your scalp completely preventing sunburn since this leaves your hair unprotected it's best to rely on a hat most of the people with textured hair fine hair or light colored hair should be especially cautious since their hair is more easily damaged by sun many weather reports tell you the day's uv index at least during the summer you can also find these predictions here for some countries including the us australia and canada here's how to interpret the international who uv index so you kn

_
Question :  ['the procedure easier for the surgeon a clean surface will provide a good view of the target area and will reduce the risk of infection take a shower and wash the area thoroughly with soap after you shave you will want to be as germ free as possible most people get anxious before getting a simple injection let alone having surgery anxiolytic drugs such as or lorazepam will greatly reduce this fear or anxiety take it as prescribed by your doctor if you do not want to take medication for your anxiety try these other relaxation deep breaths may help you to relax if you’re feeling anxious slowly breathe in through your nose and hold for two seconds then release your breath slowly out of your mouth repeat five times perform this breathing technique before you go to bed and on the morning of the procedure doing some deep breathing will stimulate your parasympathetic nervous system and help you to relax meditation is also a way to relieve anxiety close your eyes and picture you

KeyboardInterrupt: 